<a href="https://colab.research.google.com/github/gridface/ml-technical-analysis/blob/main/centaur_portfolio_projection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Portfolio Analysis with Pandas

## Define Your Variables

In [130]:
import pandas as pd, yfinance as yf, matplotlib.pyplot as plt


#here we define an array of stock tickers that will be fed into our program.
#change these tickers as desired
portfolio = ["ZIM", "T", "MOS", "VALE", "ENB", "ADM" ]

#what period do you want to evaluate?
# use "period" instead of start/end
# valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
# (optional, default is '1mo')
per = "2y"

In [133]:
stocks = yf.download(sp500,period=per)[['Adj Close','Volume']]

[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No data found, symbol may be delisted


In [136]:
#Use this for jupyter standalone
sp500 = stocks
#convert the time index to datetime type. this lets us do special date functions
sp500.to_datetime(stocks.iloc[0])
# #this step flattens out the index 
# sp500.columns = stocks.columns.to_flat_index()
sp500
# # #creates a multiindex for better sorting and grouping of data
# # # check this link for more info https://pandas.pydata.org/docs/user_guide/advanced.html
# sp500.columns = pd.MultiIndex.from_tuples(stocks.columns)

# # #lets switch levels now so we can group on individual stocks at the top level
# sp500.swaplevel(axis = 1).sort_index(axis=1)



AttributeError: 'DataFrame' object has no attribute 'to_datetime'

## Notebook Setup

In [107]:
stocks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 504 entries, 2021-02-22 00:00:00 to 2023-02-21 00:00:00
Columns: 1006 entries, ('Adj Close', 'A') to ('Volume', 'ZTS')
dtypes: float64(508), int64(498)
memory usage: 3.9+ MB


In [108]:
#this block is to point jupyter to our custom libraries
%load_ext autoreload
%autoreload 2
import os
import sys
module_path = os.path.abspath(os.path.join('../../libs'))
sys.path.insert(0, module_path)

#****************
# uncomment this block to install the yahoo finance api library for standalone and hosted jupyter notebooks. 
# !pip install yfinance
# import yfinance as yf
#****************

# once it is installed, we can import it like any other installed library
#pandas and matplotlib libraries are already part of colab, so we just import them


#to run the full app, pull in our custom libraries
# import libs.collector as collector
# import libs.indicators as indicators
#import libs.dbloader as dbloader

#general note on running notebooks: if you get an "undefined" type error, remember that you have to 
#run each of the cells in order sequentially first in order to "define" the variables
#and execute the commands. so if you get the "undefined" error, go back up and make sure you 
#have run all the lines that came before your troublesome line of code


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Get Your Portfolio Data

In [101]:
#notice the format of this data - it has a two-tiered header - [measurement]/[ticker]
# we will be changing this around below to make it easier to work with

#****************
#run this codeblock if you are running jupyter notebook as a standalone
# stocks = yf.download(portfolio,period=per)
#convert the time index to datetime type. this lets us do special date functions
# pd.to_datetime(stocks.iloc[0])
#****************

# stocks = collector.get_yfinance_stocks(portfolio,per)
# stocks = collector.flatten(stocks)
# stocks

## Create Analytical Fields

In [109]:
close = stocks['Adj Close'].copy()
close

,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2021-02-22,122.032654,20.440001,159.379578,124.501358,98.542206,103.336601,120.744865,35.160000,246.021835,467.329987,...,126.750000,57.183510,49.656055,51.815701,97.550209,101.816757,153.387390,488.309998,50.579266,157.996033
2021-02-23,120.965485,20.660000,157.944519,124.363014,97.815056,103.677422,119.020638,35.150002,249.488739,467.799988,...,136.479996,58.602924,50.341915,52.226791,96.932426,101.467690,154.851059,498.019989,51.347050,157.364685
2021-02-24,123.534584,21.820000,157.906494,123.859085,100.042526,102.440727,120.577362,35.570000,252.488174,476.619995,...,134.210007,56.975327,51.850800,52.853191,99.128967,101.496773,156.592133,512.690002,53.821037,156.901031
2021-02-25,120.649277,20.860001,152.698380,119.550949,98.790726,100.152367,119.789154,35.490002,247.141769,459.160004,...,128.660004,56.625202,50.991192,51.316536,98.060112,101.516174,157.367004,497.619995,51.726208,154.365738
2021-02-26,120.619629,20.940001,152.394257,119.817726,99.168106,98.565109,118.015663,35.820000,244.337082,459.670013,...,131.729996,55.442360,49.720074,51.942940,97.628647,100.381760,155.989426,499.429993,50.399162,153.142487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-14,153.039993,16.790001,150.759995,153.199997,152.589996,157.360001,107.260002,66.279999,284.769989,377.899994,...,109.080002,67.980003,116.419998,36.990002,107.339996,130.830002,126.709999,318.760010,52.320000,171.899994
2023-02-15,151.750000,16.799999,151.289993,155.330002,151.059998,158.809998,107.279999,67.980003,287.649994,375.940002,...,108.459999,67.900002,116.070000,36.590000,107.879997,131.669998,126.510002,327.019989,52.180000,175.020004
2023-02-16,148.279999,16.389999,150.190002,153.710007,149.529999,160.470001,106.080002,67.099998,281.220001,365.160004,...,108.470001,67.260002,115.730003,35.799999,106.250000,131.720001,125.110001,333.440002,51.389999,173.929993


In [128]:
len(close.columns)


503

In [111]:
# import numpy as np

# def moving_averages(Data):
#   for i in range(len(Data.columns)):
#     try:
#       Data[str(Data.columns[i]) + '_MA10'] = Data[Data.columns[i]].rolling(10).mean()
#       Data[str(Data.columns[i]) + '_MA50'] = Data[Data.columns[i]].rolling(50).mean()
#       Data[str(Data.columns[i]) + '_MA200'] = Data[Data.columns[i]].rolling(200).mean()
#     except ValueError:
#       pass
#   return Data

# # Exponential moving average (EMA) tells us the weighted mean of the previous K data points. 
# # EMA places a greater weight and significance on the most recent data points. 
# # In python we use the Exponentially Weighted set of functions (the mean, in our case)
# def exp_moving_averages(Data):
#   for i in range(len(Data.columns)):
#     try:
#       Data[str(Data.columns[i]) + '_EMA10'] = Data[Data.columns[i]].ewm(span=10, adjust=False).mean()
#       Data[str(Data.columns[i]) + '_EMA50'] = Data[Data.columns[i]].ewm(span=50, adjust=False).mean()
#       Data[str(Data.columns[i]) + '_EMA200'] = Data[Data.columns[i]].ewm(span=200, adjust=False).mean()
#     except ValueError:
#       pass
#   return Data


# def pct_change(Data):
#   for i in range(len(Data.columns)):
#     try:
#       Data[str(Data.columns[i]) + '_pct_change'] = Data[Data.columns[i]].pct_change()
#     except ValueError:
#       pass
#   return Data

# def log_returns(Data):
#   for i in range(len(Data.columns)):
#     try:
#       Data[str(Data.columns[i]) + '_log_returns'] = np.log(Data[Data.columns[i]]/Data[Data.columns[i]].shift())
#     except ValueError:
#       pass
#   return Data

In [113]:
# ma = indicators.moving_averages(close)
# ema = indicators.exp_moving_averages(close)

# ema.tail()

In [152]:
close.index = pd.to_datetime(close.index)
type(close.index)

In [185]:
close[:"2021-02-22"]

,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2021-02-22,122.032654,20.440001,159.379578,124.501358,98.542206,103.336601,120.744865,35.16,246.021835,467.329987,...,126.75,57.18351,49.656055,51.815701,97.550209,101.816757,153.38739,488.309998,50.579266,157.996033


## Plot the Data

In [ ]:
import matplotlib.pyplot as plt
plt.style.use("seaborn")

In [ ]:
# for reference, lets list our columns
# IDEA: consider breaking our dataset into a number of new datasets for each ticker and their analysis
close.columns

In [ ]:
close.plot(figsize = (15,8 ), fontsize = 13)
plt.legend(fontsize = 13)
plt.show()

In [ ]:
# lets normalize our data so we can start on the same starting point for all the stocks
# to do this, we take the close price from the first record in the time series, then we divide all of the 
# other stock close number by that value. from there, we multiply the numbers again by 100 to get a positive number
norm = close.div(close.iloc[0]).mul(100)

In [ ]:
norm.plot(figsize = (15, 8 ), fontsize = 13)
plt.legend(fontsize = 13)
plt.show()

In [ ]:
norm.describe()

In [ ]:
#in order to get daily returns, use the shift() method
lag1 = close.shift(periods = 1)

In [ ]:
adm = close.ADM.copy().to_frame()

In [ ]:
adm["lag1"] = adm.shift(periods = 1)

In [ ]:
adm.plot(figsize = (15, 8 ), fontsize = 13)
plt.legend(fontsize = 13)
plt.show()

In [ ]:
# close["adm_diff"] = close.ADM.sub(close.ADM.shift(periods = 1)

In [ ]:
#or we can just use the diff and pct_change methods - which is supposed to do the same thing but....
adm["adm_chg_pct"] = close.ADM.pct_change()

In [ ]:

from importlib import reload
plt=reload(plt)

plt.plot(figsize = (100, 8 ), fontsize = 13)
plt.plot(adm["adm_chg_pct"])

plt.xlabel("Date")
plt.ylabel("Change Pct")
plt.title("Annual Daily Percentage Change")

# Display

plt.show()

In [ ]:
adm.head()